<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/bigdataproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile

drive.mount('/content/drive/', force_remount=False)

with zipfile.ZipFile("/content/drive/MyDrive/Dataset_small/data1.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
with open("/targetdir/data1/train/filename.ext", "r") as file:
  pass

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

torch.cuda.current_device()
torch.cuda._initialized = True


class Inception(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1_b1 = nn.Conv2d(256, 256, 1)
        self.conv2_b1 = nn.Conv2d(256, 320, 3, padding=1)
        self.conv3_b1 = nn.Conv2d(320, 320, 3, padding=1)

        self.conv1_b2 = nn.Conv2d(256, 48, 1)
        self.conv2_b2 = nn.Conv2d(48, 64, (1, 5), padding=(0, 2))
        self.conv3_b2 = nn.Conv2d(64, 64, (5, 1), padding=(2, 0))

        self.conv1_b3 = nn.Conv2d(256, 64, 1)

        self.conv1_b4 = nn.Conv2d(256, 64, 1)

    def forward(self, X):
        # branch 1
        b1 = self.conv1_b1(X)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv2_b1(b1)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv3_b1(b1)
        b1 = b1.mul(torch.sigmoid(b1))

        # branch 2
        b2 = self.conv1_b2(X)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv2_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv3_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        # branch 3
        b3 = self.conv1_b3(X)
        b3 = b3.mul(torch.sigmoid(b3))

        # branch 4
        b4 = F.max_pool2d(X, (3, 3), stride=1, padding=1)  # 2*2 的pooling拼不上
        b4 = self.conv1_b4(b4)
        b4 = b4.mul(torch.sigmoid(b4))

        return torch.cat([b1, b2, b3, b4], 1)


class LeNet_II(nn.Module):
    def __init__(self):
        super().__init__()

        # convolutional layer
        self.conv1_b1 = nn.Conv2d(1, 64, 3, padding=1)
        self.conv2_b1 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3_b1 = nn.Conv2d(128, 256, 3, padding=1)

        self.conv1_b2 = nn.Conv2d(1, 64, 3, padding=2, dilation=2)
        self.conv2_b2 = nn.Conv2d(64, 128, 3, padding=2, dilation=2)
        self.conv3_b2 = nn.Conv2d(128, 256, 3, padding=2, dilation=2)
        self.conv4_b2 = nn.Conv2d(256, 512, 3, padding=2, dilation=2)

        self.conv = nn.Conv2d(1024, 768, 3, padding=1)

        # full connect layer
        self.fc1 = nn.Linear(768 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 3926)

        # Inception 
        self.Inception = Inception()

    # forward propagation
    def forward(self, X):
        # branch 1
        b1 = self.conv1_b1(X)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv2_b1(b1)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.conv3_b1(b1)
        b1 = F.max_pool2d(b1, (2, 2), stride=2)
        b1 = b1.mul(torch.sigmoid(b1))

        b1 = self.Inception(b1)

        # branch 2
        b2 = self.conv1_b2(X)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv2_b2(b2)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv3_b2(b2)
        b2 = b2.mul(torch.sigmoid(b2))

        b2 = self.conv4_b2(b2)
        b2 = F.max_pool2d(b2, (2, 2), stride=2)
        b2 = b2.mul(torch.sigmoid(b2))

        # tail
        out = self.conv(torch.cat([b1, b2], 1))
        out = F.max_pool2d(out, (2, 2), stride=2)
        out = out.mul(torch.sigmoid(out))
        # unfold
        out = torch.flatten(out, start_dim=1)

        # full connect
        out = self.fc1(out)
        out = out.mul(torch.sigmoid(out))
        out = F.dropout(out)

        out = self.fc2(out)
        out = F.log_softmax(out, dim=-1)

        return out


# # test
# ran = torch.rand((1, 1, 64, 64))
# model = LeNet_II()
# model(ran)


DEVICE = torch.device("cuda:0")
torch.cuda.empty_cache()

# dataMatrix = np.load('./dataMatrix.npy')
labels = np.load('./labels_num.npy')

rate = 0.7
trainSet = np.load('./trainSet.npy')
trainSet = torch.from_numpy(trainSet).float()
trainLabels = torch.from_numpy(labels[0, :int(234228 * rate)]).to(DEVICE)
# del dataMatrix

MINIBATCH_SIZE = 128    # mini batch size

# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

model = LeNet_II().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Epoch = 5
model.train()
for epoch in range(Epoch):
    print("epoch %s" % (epoch+1))
    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()
        output = model(batch_x.to(DEVICE))
        loss = F.nll_loss(output, batch_y)
        loss.backward()
        optimizer.step()
        if (step + 1) % 50 == 0:
            print("    step %s: Loss %s" % (step+1, loss.item()))
        if float(loss.item()) <= 0.05:
            break

testSet = np.load('./testSet.npy')
testSet = torch.from_numpy(testSet).float().to(DEVICE)
testLabels = torch.from_numpy(labels[0, int(234228 * rate):]).to(DEVICE)

model.eval()
test_loss = 0
correct = 0

MINIBATCH_SIZE = 128    # mini batch size

# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(testSet, testLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

with torch.no_grad():
    for step, (batch_x, batch_y) in enumerate(loader):
        output = model(batch_x)
        test_loss += F.nll_loss(output, batch_y, reduction='sum').item()  # sum a batch of loss
        predict = output.max(1, keepdim=True)[1]  # find the prediction
        correct += predict.eq(batch_y.view_as(predict)).sum().item()
        print("step %s, test loss: %s" % (step, test_loss))

testSize = testLabels.shape[0]
test_loss /= testSize
print("Test: Average loss:%s, Accuracy: %s/%s (%s)"
      % (test_loss, correct, testSize, correct / testSize))

